In [1]:
## see requirements in byoml_xgboost.environment.yml and byoml_xgboost.requirements.txt
# conda install --file env/byoml_xgboost.environment.yml
# conda activate byoml_xgboost 
# pip install -r env/byoml_xgboost.requirements.txt

In [2]:
MODEL_NAME = 'xgboost-example'

## BYOML XGBoost example

In this notebook we give an example on how to create and upload your XGBoost models. To keep things simple, we will use a toy dataset.

The key take away here to create your model is to either use the booster itself or the sklearn wrapper. Both will be accepted by the BYOML service.

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(
    n_samples=2000, n_features=7, n_informative=5, n_redundant=2, weights=[0.8, 0.2],
    scale=[5, 5, 1, 1000, 200, 20, 10], shuffle=False, random_state=42
)

X_data = pd.DataFrame(X, columns=[
    'heat', 'heat2', 'dust', 'light', 'humidity', 'pressure', 'nitrogen_concentration'
])
y_data = pd.DataFrame(y, columns=['ok'])

pd.concat(
    [X_data.head(), y_data.head()], 
    axis=1, 
    keys=['X_data', 'y_data']
)

X_data                                                           \
        heat      heat2      dust        light    humidity   pressure   
0   3.214640  -3.107601  2.663515  -599.783123   67.133968 -40.880974   
1  -0.354232  14.506926 -0.832169 -1501.497188  -42.529406   8.094005   
2   4.380911  10.794966  1.566156 -1121.889667  -30.364312 -22.922713   
3   6.396967   1.283805  1.444611  -827.627508  220.112282 -37.722183   
4  13.324452  15.490906  1.597028  -673.973149   24.585876 -29.507437   

                         y_data  
  nitrogen_concentration     ok  
0              22.175197      0  
1             -13.488557      0  
2              10.072141      0  
3              14.348605      0  
4              19.428033      0

## Create the model

We can now create the model and save it:

In [5]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=123
)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [7]:
# set xgboost params
param = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'colsample_bytree': 0.3,
    'objective': 'binary:hinge'
}
num_round = 100
bst = xgb.train(param, dtrain, num_round)

## Upload to Waylay

After your model is saved, you can upload it to Waylay, using our Python SDK:

In [8]:
from waylay import WaylayClient, RestResponseError
waylay = WaylayClient.from_profile()

# the model names we use in this example
model_names = [
    model_name, probability_model_name, classifier_model_name
] = [
    MODEL_NAME, f'{MODEL_NAME}-probs', f'{MODEL_NAME}-classifier', 
]

In [9]:
# remove existing models first
import time
def cleanup():
    for name in model_names:
        try:
            waylay.byoml.model.remove(name)
            display(f'Removed: {name}')
            time.sleep(5) # it takes a few seconds until model name becomes available again
        except RestResponseError as e:
            display(f'Ignored: {e}')
cleanup()

"Ignored: ByomlActionError(404: 'Model does not exist'; DELETE 'https://byom-staging.waylay.io/models/xgboost-example')"

"Ignored: ByomlActionError(404: 'Model does not exist'; DELETE 'https://byom-staging.waylay.io/models/xgboost-example-probs')"

"Ignored: ByomlActionError(404: 'Model does not exist'; DELETE 'https://byom-staging.waylay.io/models/xgboost-example-classifier')"

In [10]:
waylay.byoml.model.upload(
    model_name=model_name, 
    trained_model=bst, 
    framework='xgboost', 
    description='xgboost example'
)

{'message': 'Model successfully uploaded'}

and check if the output is the same:

In [11]:
bst.predict(dtest)[:5]

array([0., 0., 0., 0., 1.], dtype=float32)

In [12]:
waylay.byoml.model.predict(model_name, X_test[:5])

array([0., 0., 0., 0., 1.])

## Create an XGBoost model that outputs probabilities

Sometimes you might want to work with the probabilities directly instead of using the `1` and `0` values. To achieve this we only need a small tweak in the model parameters

In [13]:
param = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'colsample_bytree': 0.3,
    'objective': 'binary:logistic', # set this one to logistic
    'eval_metric': 'logloss' # and this one to error or logloss
}

In [14]:
bst = xgb.train(param, dtrain, num_round)

In [15]:
waylay.byoml.model.upload(
    model_name=probability_model_name, 
    trained_model=bst, 
    framework='xgboost', 
    description='xgboost probabilities example'
)

{'message': 'Model successfully uploaded'}

In [16]:
bst.predict(dtest)[:5]

array([0.03388389, 0.00636836, 0.07393802, 0.41373524, 0.59628487],
      dtype=float32)

In [17]:
waylay.byoml.model.predict(probability_model_name, X_test[:5])

array([0.03388389, 0.00636836, 0.07393802, 0.41373524, 0.59628487])

## Using the XGBClassifier
Instead of directly using a booster we can also use the XGBClassifier as is shown in the example below. 
Keep in mind although the predict method will return 1's and 0's, the uploaded model will output the probabilities as the underlying Booster object is used. 
If this is unwanted behaviour you can set the objective to `binary:hinge`.

In this case we should use the Dataframes and not the DMatrix:

In [18]:
from xgboost import XGBClassifier

In [19]:
model = XGBClassifier(
    max_depth=3,
    learning_rate=0.1,
    colsample_bytree=0.3,
    eval_metric='logloss'
)

In [20]:
model.fit(X_train, y_train.values.ravel())

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [21]:
waylay.byoml.model.upload(
    model_name=classifier_model_name, 
    trained_model=model, 
    framework='xgboost', 
    description='xgboost classifier example'
)

{'message': 'Model successfully uploaded'}

And as warned for above, the local model outputs the classes, while the deployed model uses the probabilities:

In [22]:
waylay.byoml.model.predict(classifier_model_name, X_test[:5])

array([0.03388389, 0.00636836, 0.07393802, 0.41373524, 0.59628487])

## Cleanup

In [23]:
cleanup()

'Removed: xgboost-example'

'Removed: xgboost-example-probs'

'Removed: xgboost-example-classifier'